In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from pylab import rcParams
rcParams['figure.figsize'] = 12,8

In [3]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

import re

import os
from glob import glob

import tensorflow as tf
import keras

In [4]:
import warnings
warnings.simplefilter(action="ignore")

In [6]:
date_parse = lambda dates: pd.to_datetime(dates)
path = f"/diploma_info/datalake/processed_data/GOOGLE_daily.csv"

full_set = pd.read_csv(
    path,
    parse_dates=["date"],
    date_parser=date_parse,
    index_col=["date"],
)

In [9]:
full_set['close_max_3_days_d-1'] = full_set['close_max_3_days'].shift(1).fillna(0)
full_set['close_min_3_days_d-1'] = full_set['close_min_3_days'].shift(1).fillna(0)
full_set['close_mean_3_days_d-1'] = full_set['close_mean_3_days'].shift(1).fillna(0)

In [16]:
X_train = full_set.loc[:'2024-01-01', ['close_lag_1', 'close_max_3_days_d-1', 'close_min_3_days_d-1', 'close_mean_3_days_d-1']].values
X_test = full_set.loc['2024-01-01':'2024-02-08', ['close_lag_1', 'close_max_3_days_d-1', 'close_min_3_days_d-1', 'close_mean_3_days_d-1']].values
y_train = full_set.loc[:'2024-01-01', 'growth'].values
y_test = full_set.loc['2024-01-01':'2024-02-08', 'growth'].values


X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [21]:
num_classes = len(np.unique(y_train))

In [22]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=X_train.shape[1:])

In [24]:
epochs = 500
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.keras", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

Epoch 1/500


122/122 [==============================] - 4s 13ms/step - loss: 0.7031 - sparse_categorical_accuracy: 0.5041 - val_loss: 0.7451 - val_sparse_categorical_accuracy: 0.5344 - lr: 0.0010
Epoch 2/500
122/122 [==============================] - 1s 6ms/step - loss: 0.6983 - sparse_categorical_accuracy: 0.5090 - val_loss: 0.6956 - val_sparse_categorical_accuracy: 0.5344 - lr: 0.0010
Epoch 3/500
122/122 [==============================] - 1s 5ms/step - loss: 0.6964 - sparse_categorical_accuracy: 0.4972 - val_loss: 0.7252 - val_sparse_categorical_accuracy: 0.5344 - lr: 0.0010
Epoch 4/500
122/122 [==============================] - 1s 5ms/step - loss: 0.6979 - sparse_categorical_accuracy: 0.4992 - val_loss: 1.2395 - val_sparse_categorical_accuracy: 0.4656 - lr: 0.0010
Epoch 5/500
122/122 [==============================] - 1s 5ms/step - loss: 0.6952 - sparse_categorical_accuracy: 0.5010 - val_loss: 0.9307 - val_sparse_categorical_accuracy: 0.5344 - lr: 0.0010
Epoch 6/500
122/122 [======

122/122 [==============================] - 1s 6ms/step - loss: 0.6930 - sparse_categorical_accuracy: 0.5154 - val_loss: 0.7834 - val_sparse_categorical_accuracy: 0.4656 - lr: 5.0000e-04
Epoch 41/500
122/122 [==============================] - 1s 6ms/step - loss: 0.6934 - sparse_categorical_accuracy: 0.5056 - val_loss: 0.9565 - val_sparse_categorical_accuracy: 0.4656 - lr: 5.0000e-04
Epoch 42/500
122/122 [==============================] - 1s 6ms/step - loss: 0.6932 - sparse_categorical_accuracy: 0.5131 - val_loss: 0.8596 - val_sparse_categorical_accuracy: 0.5344 - lr: 5.0000e-04
Epoch 43/500
122/122 [==============================] - 1s 6ms/step - loss: 0.6933 - sparse_categorical_accuracy: 0.5172 - val_loss: 0.7494 - val_sparse_categorical_accuracy: 0.4656 - lr: 5.0000e-04
Epoch 44/500
122/122 [==============================] - 1s 8ms/step - loss: 0.6920 - sparse_categorical_accuracy: 0.5169 - val_loss: 0.7441 - val_sparse_categorical_accuracy: 0.4656 - lr: 5.0000e-04
Epoch 45/500
122/1

122/122 [==============================] - 1s 6ms/step - loss: 0.6915 - sparse_categorical_accuracy: 0.5141 - val_loss: 0.7009 - val_sparse_categorical_accuracy: 0.5344 - lr: 1.2500e-04
Epoch 82/500
122/122 [==============================] - 1s 6ms/step - loss: 0.6913 - sparse_categorical_accuracy: 0.5231 - val_loss: 0.6952 - val_sparse_categorical_accuracy: 0.4944 - lr: 1.2500e-04
Epoch 83/500
122/122 [==============================] - 1s 6ms/step - loss: 0.6921 - sparse_categorical_accuracy: 0.5172 - val_loss: 0.7049 - val_sparse_categorical_accuracy: 0.4769 - lr: 1.2500e-04
Epoch 84/500
122/122 [==============================] - 1s 6ms/step - loss: 0.6910 - sparse_categorical_accuracy: 0.5236 - val_loss: 0.6931 - val_sparse_categorical_accuracy: 0.5354 - lr: 1.2500e-04
Epoch 85/500
122/122 [==============================] - 1s 6ms/step - loss: 0.6912 - sparse_categorical_accuracy: 0.5233 - val_loss: 0.6969 - val_sparse_categorical_accuracy: 0.5344 - lr: 1.2500e-04
Epoch 86/500
122/1

122/122 [==============================] - 1s 6ms/step - loss: 0.6907 - sparse_categorical_accuracy: 0.5292 - val_loss: 0.6915 - val_sparse_categorical_accuracy: 0.5344 - lr: 1.0000e-04
Epoch 123/500
122/122 [==============================] - 1s 6ms/step - loss: 0.6905 - sparse_categorical_accuracy: 0.5292 - val_loss: 0.6985 - val_sparse_categorical_accuracy: 0.4759 - lr: 1.0000e-04
Epoch 124/500
122/122 [==============================] - 1s 6ms/step - loss: 0.6910 - sparse_categorical_accuracy: 0.5210 - val_loss: 0.7103 - val_sparse_categorical_accuracy: 0.5333 - lr: 1.0000e-04
Epoch 125/500
122/122 [==============================] - 1s 8ms/step - loss: 0.6908 - sparse_categorical_accuracy: 0.5267 - val_loss: 0.6985 - val_sparse_categorical_accuracy: 0.4831 - lr: 1.0000e-04
Epoch 125: early stopping


In [26]:
y_pred = []

model = keras.models.load_model("best_model.keras")

test_loss, test_acc = model.evaluate(X_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

1/1 [==============================] - 0s 403ms/step - loss: 0.6924 - sparse_categorical_accuracy: 0.5185
Test accuracy 0.5185185074806213
Test loss 0.6924479603767395
